In [7]:
import kagglehub
import os

path = kagglehub.dataset_download("ravi72munde/uber-lyft-cab-prices")
print(os.listdir(path))
print("Path to dataset files:", path)

['weather.csv', 'cab-weather data', 'cab_rides.csv', 'Cab-Weather Data']
Path to dataset files: /root/.cache/kagglehub/datasets/ravi72munde/uber-lyft-cab-prices/versions/4


In [14]:
import pandas as pd
df=pd.read_csv(os.path.join(path, "cab_rides.csv"))
df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL
